In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Crew, Task

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "MYKEY"
os.environ["OPENAI_BASE_URL"] = "https://openrouter.ai/api/v1"
os.environ["OPENAI_MODEL_NAME"] = "mistralai/mistral-7b-instruct:free"
os.environ["SERPER_API_KEY"] = "MYKEY"

In [4]:
from langchain_openai import ChatOpenAI

In [5]:
llm = ChatOpenAI(
    model="mistralai/mistral-7b-instruct:free",  # Free model
    temperature=0,
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_base=os.environ["OPENAI_BASE_URL"],
    max_tokens=300  # ✅ This keeps responses short so you don’t burn credits
)


In [6]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [7]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    llm=llm,
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [8]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    llm=llm,
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [9]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    llm=llm,
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

In [10]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [11]:
venue_task = Task(
    description=(
        "Find a venue in {event_city} that meets criteria for {event_topic}. "
        "Search online for venues, and for each venue you find, "
        "use the website content scraping tool one by one to get detailed information. "
        "Do not use multiple website URLs in one tool call — read them one at a time."
    ),
    expected_output="All the details of a specifically chosen venue you found to accommodate the event.",
    human_input=False,
    output_json=VenueDetails,
    output_file="venue_details.json",
    agent=venue_coordinator
)


In [12]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [13]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

In [14]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

In [15]:
import json

# Sample venue data
venue_data = {
    "venues": [
        {
            "name": "Grand Hall",
            "location": "New Delhi",
            "capacity": 300,
            "price_per_day": 50000
        },
        {
            "name": "Eco Garden",
            "location": "Bangalore",
            "capacity": 150,
            "price_per_day": 30000
        }
    ]
}

# Save to venue_details.json
with open("venue_details.json", "w") as f:
    json.dump(venue_data, f, indent=4)

print("✅ venue_details.json file created successfully.")


✅ venue_details.json file created successfully.


In [16]:
!type venue_details.json


{
    "venues": [
        {
            "name": "Grand Hall",
            "location": "New Delhi",
            "capacity": 300,
            "price_per_day": 50000
        },
        {
            "name": "Eco Garden",
            "location": "Bangalore",
            "capacity": 150,
            "price_per_day": 30000
        }
    ]
}


In [17]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [18]:
# 🟢 Kick off the crew with event details
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference. Search online for venues, and for each venue you find, use the website content scraping tool one by one to get detailed information. Do not use multiple website URLs in one tool call — read them one at a time.


> Entering new CrewAgentExecutor chain...
 Action: Search the internet
Action Input: {'search_query': 'Tech Innovation Conference venues San Francisco'} 


Search results: Title: The Best 12 Tech Conferences to Attend in San Francisco 2025
Link: https://vanguard-x.com/tech-events-2025/tech-conferences-san-francisco-2025/
Snippet: Venue: Grand Hyatt SFO  TECHSPO brings together a spectrum of technology makers and marketers in one of the city's most accessible venues. This event offers an interactive platform for showcasing new solutions in SaaS, adtech, martech, mobile, and connected devices.
---
Title: TECHSPO San Francisco 2

Exception in thread Thread-6 (_execute):
Traceback (most recent call last):
  File "C:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\hp\crew-env\Lib\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\hp\crew-env\Lib\site-packages\crewai\task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\hp\crew-env\Lib\site-packages\crewai\agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\hp\crew-env\Lib\site-packages\langchain\chains\base.py", line 163, in invoke
    raise e
  File "C:\Users\hp\crew-env\Lib\site-packages\langchain\chains\base.py", line 153, in invoke
    

In [21]:
from IPython.display import Markdown
Markdown("marketing_report.md")

---

# Marketing Activities for Tech Innovation Conference 2035

## Social Media Promotion

* Posting event details and highlights on multiple platforms, including Facebook, Twitter, LinkedIn, and Instagram.
* Utilizing relevant hashtags to reach a wider audience: #TechInnovation2035, #InnovationConference, #TechEvent, #SanFrancisco, etc.
* Collaborating with industry influencers and thought leaders to create engaging content and increase event exposure.

## Email Marketing

* Sending targeted emails to an audience of potential attendees interested in technology and innovation.
* Incorporating eye-catching visuals, concise messaging, and a clear call-to-action to encourage registration.
* Segmenting the email list based on specific interests, past attendance, and demographics for personalized and effective outreach.

## Partnerships and Sponsorships

* Collaborating with technology companies, associations, and media outlets to amplify the event's reach and attract more attendees.
* Offering various levels of sponsorship packages, including brand visibility, product showcases, and networking opportunities:
	+ [Sponsor 1]
	+ [Sponsor 2]
	+ [Sponsor 3]
* Leveraging partnerships to secure keynote speakers, panelists, and exhibitors that draw a larger crowd:
	+ [Speaker 1]
	+ [Speaker 2]
	+ [Speaker 3]
	+ [Panelist 1]
	+ [Panelist 2]
	+ [Panelist 3]
	+ [Exhibitor 1]
	+ [Exhibitor 2]
	+ [Exhibitor 3]

## Attendee Engagement

* Creating an interactive event app to facilitate networking, schedule building, and access to event materials.
* Organizing a variety of activities, such as workshops, Q&A sessions, and networking events, to encourage attendee interaction and engagement:
	+ [Workshop 1]
	+ [Workshop 2]
	+ [Workshop 3]
	+ [Q&A Session 1]
	+ [Q&A Session 2]
	+ [Q&A Session 3]
	+ [Networking Event 1]
	+ [Networking Event 2]
	+ [Networking Event 3]
* Encouraging attendees to share their experiences on social media using event-specific hashtags and tagging the event handle.

## Event Updates and Reminders

* Sending out regular updates and reminders to registered attendees, including event agenda, speaker lineup, and any changes or important information.
* Utilizing push notifications and email marketing to keep attendees informed and excited about the event.

---